In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

def generate_text(prompt, model_name='gpt2', max_length=100):
    # Load pre-trained model tokenizer (vocabulary)
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)

    # Encode the input text to tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode(prompt, return_tensors='pt')

    # Load pre-trained model
    model = GPT2LMHeadModel.from_pretrained(model_name)

    # Generate text until the output length (which includes the context length) reaches 100
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)

    # Decode the output tensor to a string
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text

In [6]:
prompt = "WHo was last president of usa"
print(generate_text(prompt))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


WHo was last president of usa- tion. He was a man of great integrity and integrity. He was a man of great integrity and integrity. He was a man of great integrity and integrity. He was a man of great integrity and integrity. He was a man of great integrity and integrity. He was a man of great integrity and integrity. He was a man of great integrity and integrity. He was a man of great integrity and integrity. He was a man of great integrity and integrity
